In [1]:
from mutagen.mp4 import MP4
from pathlib import Path
import re

In [2]:
input_path = Path.cwd() / 'input'

In [3]:
input_folder = 'Watch - TV Shows (Anime)'
input_path = Path('d:/') / input_folder # f in asus, d in lenovo

In [4]:
folder_list = [x for x in input_path.iterdir() if x.is_dir()]

In [5]:
def update_title(file_path):
    file = MP4(file_path)
    title = ''.join(file['©nam'])
    
    if not title:
        file['©nam'] = file_path.stem
        file['©cmt'] = file_path.stem
        file.save()

In [6]:
def get_season(folder_path):
    pattern = r'\[S\d+\]'
    result = re.search(pattern, folder_path.name)
    if result:
        text = result.group()
        search = re.search(r'\d+', text)
        season = search.group().zfill(2)
    else:
        season = '1'.zfill(2)
    return season

In [7]:
def get_episode(file_path):
    patterns = [
        r'\[E\d+\]',
        r'\[\d+(v\d)?\]',
        r'\[\d+\s?END\]',
        r'\[\d+_\d+\]',
        r'-\s?\d+',
        r'\.\d+',
        r'第\d+话',
        r'第\d+話'
    ]
    
    for pattern in patterns:
        result = re.search(pattern, file_path.stem)
        if result:
            text = result.group()
            search = re.search(r'\d+', text)
            episode = search.group()
            return episode

In [8]:
def get_episodes(file_list):
    episodes = []
    for file_path in file_list:
        keywords = ['.5', 'OVA']
        for keyword in keywords:
            if keyword in file_path.stem:
                episode = None
                break
        else:
            episode = get_episode(file_path)
        episodes.append(episode)
    
    num_digit = max([len(x) for x in episodes if x])
    episodes = [x.zfill(num_digit) if x else x for x in episodes]
    return episodes

In [9]:
for folder_path in folder_list:
    file_list = list(folder_path.glob('*.mp4'))
    for file_path in file_list:
        update_title(file_path)
    
    pattern = r'\[(.*?)\]'
    result = re.findall(pattern, folder_path.name)
    english = result[0]
    chinese = result[1]
    subtitle = result[-3]
    resolution = result[-2]

    print(folder_path.name)
    season = get_season(folder_path)
    episodes = get_episodes(file_list)
    for episode in episodes:
        if episode:
            name = f'{english}.{chinese}.S{season}E{episode}.{subtitle}.{resolution}.mp4'
            print(name)
    print()

[86_Eight-Six][不存在的战区][01-23][CHS][1080P][MP4]
86_Eight-Six.不存在的战区.S01E01.CHS.1080P.mp4
86_Eight-Six.不存在的战区.S01E02.CHS.1080P.mp4
86_Eight-Six.不存在的战区.S01E03.CHS.1080P.mp4
86_Eight-Six.不存在的战区.S01E04.CHS.1080P.mp4
86_Eight-Six.不存在的战区.S01E05.CHS.1080P.mp4
86_Eight-Six.不存在的战区.S01E06.CHS.1080P.mp4
86_Eight-Six.不存在的战区.S01E07.CHS.1080P.mp4
86_Eight-Six.不存在的战区.S01E08.CHS.1080P.mp4
86_Eight-Six.不存在的战区.S01E09.CHS.1080P.mp4
86_Eight-Six.不存在的战区.S01E10.CHS.1080P.mp4
86_Eight-Six.不存在的战区.S01E11.CHS.1080P.mp4
86_Eight-Six.不存在的战区.S01E12.CHS.1080P.mp4
86_Eight-Six.不存在的战区.S01E13.CHS.1080P.mp4
86_Eight-Six.不存在的战区.S01E14.CHS.1080P.mp4
86_Eight-Six.不存在的战区.S01E15.CHS.1080P.mp4
86_Eight-Six.不存在的战区.S01E16.CHS.1080P.mp4
86_Eight-Six.不存在的战区.S01E17.CHS.1080P.mp4
86_Eight-Six.不存在的战区.S01E18.CHS.1080P.mp4
86_Eight-Six.不存在的战区.S01E19.CHS.1080P.mp4
86_Eight-Six.不存在的战区.S01E20.CHS.1080P.mp4
86_Eight-Six.不存在的战区.S01E21.CHS.1080P.mp4
86_Eight-Six.不存在的战区.S01E22.CHS.1080P.mp4
86_Eight-Six.不存在的战区.S01E23.CHS.1080P.mp4

[Ace_of_D